# Engines

In [ ]:
%run Utils/Utils_ContinuousMovement.ipynb
import numpy as np

In [ ]:
# Create Node to communicate with ROS
rospy.init_node('sub_odom')
dx,dy,angle = 0,0,0
real_angle, real_x, real_y = 0,0,0
moved_distance = 0
rotations,movements,observation = [],[],[]

# Create a publisher to control movement
pub = rospy.Publisher('cmd_vel', Twist, queue_size=1)

# Run Thread
running_thread = False
if not running_thread:
    servoThread = Thread(target = measurementThread)
    servoThread.start()
    print('Servo Motors Ready!')

# Lidar

In [ ]:
%run Utils/Utils_Lidar.ipynb

In [ ]:
# Start Lidar
from rplidar import RPLidar
import rplidar

lidar = RPLidar('/dev/ttyUSB0')
lidar.motor_speed = rplidar.MAX_MOTOR_PWM

lidarThread = Thread(target = readLidarThread)
lidarThread.start()

print("Lidar has started streaming data!")

# Faster RCNN

In [ ]:
%run Utils/Utils_FasterRCNN.ipynb
print("Object recognition model Ready!")

## Get Actions

In [ ]:
def GetAction(agent):
    global observation
    observation = torch.Tensor(observation).cuda()
    dists = agent.get_dists([observation], [], masks=torch.ones((1,6)))
    actions = []
    for dist in dists[0]:
        actions.append(dist.sample().cpu().numpy()[0][0])
    return actions

In [ ]:
def GetAction_mod(agent):
    global observation
    observation = torch.Tensor(observation).cuda()
    hidden,_ = agent.network_body(vis_inputs=[0],vec_inputs=[observation])
    dists = agent.distribution(hidden, masks = torch.ones((1, 6)))
    actions = []
    for dist in dists:
        actions.append(dist.sample().cpu().numpy()[0][0])
    return actions

## Get Observation

In [1]:
import math,copy
def getObservation():
    # Get Lidar Readings
    global real_x, real_y, clockIsVisible, observation, realAngles, readings, old_readings

    # Lidar Readings
    previous_readings = {k:round(readings[k]/1000,3) if readings[k]<=8000 else 8.000 for k in realAngles}

    if old_readings == None:
        old_readings = copy.deepcopy(previous_readings)
    else:
        for k in previous_readings.keys():
            new_angles = [previous_readings[k]+y for y in range(-5,6)]
            new_angle = min([x for x in new_angles if x != old_readings[k]  and x>0.01])
            previous_readings[k]=new_angle
        old_readings = copy.deepcopy(previous_readings)
    
            
    # Angle & DX/DY
    angle = calculateAngleToGoal()
    dx = goal_x_coor - real_x
    dy = goal_y_coor - real_y
    
    # Dist to Goal
    distToGoal = math.sqrt(dx**2+dy**2)
    
    # Observation
    observation = [round(distToGoal,3), round(dx,3), round(dy,3), angle, clockIsVisible]+list(previous_readings.values())

In [ ]:
def act(actions):
    
    # Actions
    movement = actions[0]
    rotation = actions[1]
    
    # Act
    if movement == 1:  #FW
        Move(distance=0.10, speed=0.15)
        update_coordinates()
        
    elif movement == 2:
        Move(distance=0.10, speed=-0.15)
        update_coordinates()
    
    # Rotation
    if rotation == 1:  # Right
        Rotate(degrees=10, direction='right')
    
    elif rotation == 2:  # Left
        Rotate(degrees=10, direction='left')

In [ ]:
def calculateAngleToGoal():
    
    global real_angle
    
    relAngToGoal = relativeAngleToGoal()
    
    # If goal is to the right, either up or down
    if relAngToGoal > 0 and relAngToGoal < 180:
        
        # case 1
        if (relAngToGoal - real_angle) > -180:
            return round((relAngToGoal - real_angle), 3) 

        # case 2:
        elif (relAngToGoal - real_angle) < -180:
            return round((relAngToGoal - (real_angle-360)),3)
    
    
    # 
    elif relAngToGoal < 0:
        relAngToGoal+=360
        
        # case 3
        if (relAngToGoal - real_angle) > 180:
            return round((relAngToGoal - 360 - real_angle), 3)
        
        elif (relAngToGoal - real_angle) < 180:
            return round((relAngToGoal - real_angle),3)

In [ ]:
# Relative to Vertical
def relativeAngleToGoal():
    
    global real_x, real_y
    global goal_x_coor, goal_y_coor
    
    dis_to_goal_x = goal_x_coor - real_x
    dis_to_goal_y = goal_y_coor - real_y
    
    # Left Side
    if dis_to_goal_x < 0:
        
        # If Up
        if dis_to_goal_y>0:
            return round(math.degrees(math.atan(dis_to_goal_x/dis_to_goal_y)),3)
        
        # If Down
        else:
            return round(math.degrees(math.atan(dis_to_goal_x/dis_to_goal_y)),3)-180
                
    # Right Side
    else:
        # If Up
        if dis_to_goal_y>0:
            return round(math.degrees(math.atan(dis_to_goal_x/dis_to_goal_y)),3)
        
        # If Down
        else:
            return 180+round(math.degrees(math.atan(dis_to_goal_x/dis_to_goal_y)),3)